# 資料視覺化 
* [Pandas](http://pandas.pydata.org)：讀取資料與整理數據。
* [Bokeh](http://bokeh.pydata.org/en/latest/)：Bokeh是一種互動式資料視覺化的Python套件，並可以在瀏覽器上呈現，畫面優雅簡潔。
-----
# 1. Bokeh基本圖型
* 圖表（Charts）：一個高級接口（high-level interface），用以簡單快速地建立復雜的統計圖表。

* 繪圖（Plotting）：一個中級接口（intermediate-level interface），以構建各種視覺符號為核心。

* 模塊（Models）：一個低級接口（low-level interface），為應用程序開發人員提供最大的靈活性。

-----
# 2. 應用範例：
## (1) 讀取網頁表格
* pandas讀取及整理[台銀匯率](http://rate.bot.com.tw/xrt?Lang=zh-TW)
* 利用Bokeh將歷史匯率畫成折線圖
-----

## (2)讀取Excel檔案
* pandas讀取及整理[農產品批發市場交易行情站](http://amis.afa.gov.tw/main/Main.aspx)下載的Excel檔案
* 利用Bokeh繪圖

-----
## (3) Bokeh server上做圖表互動

-----

## <font color="DEEPPINK">一開始，我們先來認識幾個Bokeh基本圖型</font>
## <font color="DEEPPINK">試試看先長一條線</font>

In [2]:
import numpy as np
from bokeh.charts import Line, show, output_notebook

xyvalues = np.array([[2, 3, 7, 5, 26]])
line = Line(xyvalues, title="line", legend="top_left", ylabel='Languages')
output_notebook()
show(line)

Loading BokehJS ...

## <font color="DEEPPINK">長多條線看看...</font>

In [3]:
xyvalues = np.random.randint(0,30,(3, 6))
line = Line(xyvalues, title="line", legend="top_left", ylabel='Languages')
output_notebook()
show(line)

Loading BokehJS ...

## <font color="DEEPPINK">換個方式，用中級接口畫圖</font>

In [4]:
from bokeh.plotting import figure
x= [1,2,3,4,5]
top= [6,7,8,9,10]
p = figure( title="自訂標題", x_axis_label = '自訂x座標', y_axis_label = '自訂y座標' )
p.vbar(x=x, width=0.5, bottom=0, top=top, color="#CAB2D6")

show(p)

## <font color="DEEPPINK">每一個點可以換樣式</font>

In [5]:
from bokeh.plotting import figure
from bokeh.layouts import row

p1 = figure(width=300, height=300,title="自訂標題", x_axis_label = '自訂x座標', y_axis_label = '自訂y座標')
p1.asterisk(x=[1,2,3], y=[1,2,3], size=20, color="#F0027F")

p2 = figure(plot_width=300, plot_height=300)
p2.patches(xs=[[1,2,3],[4,5,6,5]], ys=[[1,2,1],[4,5,5,4]],
         color=["#43a2ca", "#a8ddb5"])

p3 = figure(width=300, height=300)
p3.ellipse(x=[1, 2, 3], y=[1, 2, 3], width=30, height=30,
             color="#386CB0", fill_color=None, line_width=2)

show(row(p1, p2, p3))

##  <font color="DEEPPINK">熱身完，開始進入報告主題</font>
> ###  <font color="grey">抓取台銀網站各幣別的匯率歷史資料</font>

##  <font color="DEEPPINK">打開terminal 安裝 html5lib：conda install -c anaconda html5lib</font>

In [1]:
import pandas as pd

df_day = pd.read_html('http://rate.bot.com.tw/xrt?Lang=zh-TW')[0]

In [2]:
currencies = df_day['幣別'].str.split(' ').str.get(1).str.strip('()')[:10]

## <font color="DEEPPINK">先試一個幣別</font>

In [3]:
#df = pd.read_html('http://rate.bot.com.tw/xrt/quote/l6m/USD')[0] 
df = pd.read_html('http://rate.bot.com.tw/xrt/quote/l6m/USD')[0].iloc[:, 0:6]
header =['掛牌日期', '幣別','現金買入','現金賣出','即期買入','即期賣出']
df.columns = header

In [8]:
from bokeh.charts import output_notebook, show, output_file
from bokeh.plotting import Figure,ColumnDataSource
output_notebook()

TOOLS = ['box_zoom', 'box_select', 'wheel_zoom', 'reset', 'pan', 'resize', 'save']
f = Figure(plot_width=800, plot_height=300, x_axis_type="datetime", x_axis_label='日期', y_axis_label='價格',tools=TOOLS)
f.line(pd.to_datetime(df['掛牌日期']), df['現金賣出'], legend="USD", line_width=3, line_color='red')
show(f)

Loading BokehJS ...

## <font color=HOTPINK>多種幣別</font>

In [9]:
from bokeh.models import HoverTool
from bokeh.palettes import Spectral11
output_notebook()

df_day = pd.read_html('http://rate.bot.com.tw/xrt?Lang=zh-TW')[0]
header =['掛牌日期', '幣別','現金買入','現金賣出','即期買入','即期賣出']
currencies = df_day['幣別'].str.split(' ').str.get(1).str.strip('()')[:10]

TOOLS = ['box_zoom', 'box_select', 'wheel_zoom', 'reset', 'pan', 'resize', 'save', 'hover']
p1 = Figure(plot_width=800, plot_height=600, x_axis_type="datetime", x_axis_label='日期', y_axis_label='價格',tools=TOOLS)

myColors = Spectral11[0:len(currencies)]
baseUrl = 'http://rate.bot.com.tw/xrt/quote/l6m/{}'

index=0
for currency in currencies:
    url = baseUrl.format(currency)
    df = pd.read_html(url)[0].iloc[:, 0:6]
    df.columns = header
    p1.line(pd.to_datetime(df['掛牌日期']), df['即期買入'], legend=currency, line_width=3, line_color=myColors[index])
    index += 1

hover = p1.select(dict(type=HoverTool))
hover.tooltips = """<div>日期: @x</div>
<div>即期買入: @y</div>
"""

show(p1)

Loading BokehJS ...

## <font color=HOTPINK>還有時間的話，我們繼續講匯入EXCEL檔案產生圖表</font>

In [4]:
df = pd.read_excel("fruit_price.xls", sheetname=None)

## <font color=HOTPINK>我們先抓一個Sheet</font>

In [12]:
from bokeh.plotting import Figure, ColumnDataSource
from bokeh.models import HoverTool, NumeralTickFormatter

df_pineapple = pd.read_excel("fruit_price.xls", sheetname="pineapple")
datasource = ColumnDataSource(dict(
        date=df_pineapple['date'],
        amt=[str(x*y) for x,y in zip(df_pineapple['QTY'], df_pineapple['avg_price'])],
        price=df_pineapple['avg_price'],
        qty=df_pineapple['QTY'],
        tipdate=[x.strftime("%Y-%m-%d") for x in df_pineapple['date']]
    ))
TOOLS = ['box_zoom', 'box_select', 'wheel_zoom', 'reset', 'pan', 'resize', 'save']
hover = HoverTool(
        tooltips=[
            ("DATE", "@tipdate"),
            ("AVG_PRICE","@price"),
            ("QTY", "@qty"),
            ("AMOUNT", "@amt")
        ]
    )
f1 = Figure(plot_width=800, plot_height=600, x_axis_type="datetime", x_axis_label='日期', y_axis_label='交易量 x 價格',tools=TOOLS + [hover])
f1.line('date','amt', line_width=3, source = datasource)
f1.yaxis.formatter = NumeralTickFormatter(format="0,000")
show(f1)

## <font color=HOTPINK>還有時間嗎？接下來我們用下拉式選單做互動，先長一個選單</font>

In [13]:
from bokeh.models.widgets import Select

fruit=[("pineapple","鳳梨"),("strawberry","草莓")]
select = Select(title="水果", value="pineapple", options=fruit)

show(select)

## <font color=HOTPINK>成功了！繼續加入其他內容</font>

In [14]:
import pandas as pd
from bokeh.charts import Line, show, output_notebook
from bokeh.models.widgets import Select
from bokeh.plotting import Figure, ColumnDataSource
from bokeh.layouts import column, row
from bokeh.models import HoverTool, NumeralTickFormatter

fruit=[("pineapple","鳳梨"),("strawberry","草莓")]
select = Select(title="水果", value="pineapple", options=fruit)

output_notebook()
df = pd.read_excel('fruit_price.xls',sheetname=None)

def select_fruit(fruit_val):
    return dict(
        date = df[fruit_val]['date'],
        amt = [str(x*y) for x,y in zip(df[fruit_val]['QTY'], df[fruit_val]['avg_price'])],
        price=df[fruit_val]['avg_price'],
        qty=df[fruit_val]['QTY'],
        tipdate=[x.strftime("%Y-%m-%d") for x in df[fruit_val]['date']]
    )
source = ColumnDataSource(data=select_fruit(select.value))

def update(attr, old, new):
    source.data = select_fruit(select.value)
select.on_change('value', update)    

TOOLS = ['box_zoom', 'box_select', 'wheel_zoom', 'reset', 'pan', 'resize', 'save']
hover = HoverTool(
        tooltips=[
            ("DATE", "@tipdate"),
            ("AVG_PRICE","@price"),
            ("QTY", "@qty"),
            ("AMOUNT", "@amt")
        ]
    )

f2 = Figure(plot_width=800, plot_height=600, x_axis_type="datetime", x_axis_label='日期', y_axis_label='交易量 x 價格',tools=TOOLS +[hover])
f2.line('date','amt', line_width=3, source = source)
f2.yaxis.formatter = NumeralTickFormatter(format="0,000")

layout = column(row(select, width=400), row(f2))
show(layout)

Loading BokehJS ...

## <font color=HOTPINK>糟糕！怎麼沒反應？？</font>